## Custom Dataset Class for Loading MNIST Data

This code snippet defines a Python class, `Dataset`, responsible for loading the MNIST training and testing data from the filesystem. The MNIST data files are expected to be in the IDX file format.

1. **Import Dependencies**: The required modules, `numpy as np` and `struct`, are imported at the beginning of the code to handle array operations and binary data reading, respectively.

2. **Initialization**: The `__init__` method initializes the class and immediately calls methods to load the training and testing labels and images.

3. **Reading Labels**: The `read_idx_labels` method reads the labels from an IDX formatted file. It reads the magic number and the number of items from the file header and then loads the labels into a NumPy array.

4. **Reading Images**: Similar to `read_idx_labels`, the `read_idx_images` method reads image data from an IDX formatted file. It reads the magic number, the number of items, and the dimensions (rows and cols) of each image. The image data is loaded into a 3D NumPy array and normalized by dividing by 255.

5. **Get Train and Test Data**: The `get_train_test_data` method returns the loaded training and testing image and label datasets.

6. **Instantiation and Data Retrieval**: Finally, an instance of the `Dataset` class is created, and the training and testing data are retrieved using `get_train_test_data`.

Note: While making `Dataset` a class might seem like overkill for this simple example, this approach stems from a larger project where the class had additional features and functionalities.

In [5]:
import struct
import numpy as np

class Dataset(object):
    def __init__(self) -> None:
        self.train_labels = self.read_idx_labels("data/train-labels.idx1-ubyte")
        self.train_images = self.read_idx_images("data/train-images.idx3-ubyte")
        self.test_labels = self.read_idx_labels("data/t10k-labels.idx1-ubyte")
        self.test_images = self.read_idx_images("data/t10k-images.idx3-ubyte")
        
    def read_idx_labels(self, file_path : str) -> np.ndarray:
        with open(file_path, 'rb') as f:
            magic, num = struct.unpack(">II", f.read(8))
            labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels

    def read_idx_images(self, file_path : str) -> np.ndarray:
        with open(file_path, 'rb') as f:
            magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
            images = np.frombuffer(f.read(), dtype=np.uint8).reshape(num, rows, cols)
        return images.astype('float32')/255
    
    def get_train_test_data(self):
        return self.train_images, self.train_labels, self.test_images, self.test_labels

data = Dataset()
X_train, y_train, X_test, y_test = data.get_train_test_data()

## Creating a PyTorch Neural Network

This section demonstrates how to create a neural network model using PyTorch to solve the MNIST digit classification problem.

1. **Import Dependencies**: We import the necessary PyTorch modules such as `torch`, `nn`, and `optim`.

2. **Define the Model Class**: We create a PyTorch model class named `MyModel` that inherits from `nn.Module`. Inside the class, we:
    - Define the layers in the `__init__` method. 
    - Implement the `forward` method to describe how data flows through the network.

3. **Layers**:
    - `Flatten`: To flatten the 28x28 input images.
    - `fc1`: A fully connected layer with 128 units and ReLU activation.
    - `fc2`: Another fully connected layer with 64 units and ReLU activation.
    - `fc3`: The output fully connected layer with 10 units, corresponding to 10 classes. Softmax is applied in the `forward` method.

4. **Model Initialization**: An instance of the `MyModel` class is created.

5. **Optimizer and Loss Function**: 
    - `Adam` optimizer is used for optimizing the model parameters.
    - `CrossEntropyLoss` is chosen as the loss function, suitable for multi-class classification tasks.

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = self.flatten(x)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.softmax(self.fc3(x), dim=1)
        return x

model = MyModel()
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()


## Training the PyTorch Model

This section outlines the steps for training the PyTorch model on the MNIST dataset.

1. **Convert Labels to Torch Tensors**: We convert the `y_train` and `y_test` labels to long PyTorch tensors.

2. **Create Data Loaders**: 
    - The training dataset and dataloader are created using PyTorch's `TensorDataset` and `DataLoader` classes.
    - Batch size is set to 128 and the dataset is shuffled before each epoch.

3. **Training Loop**:
    - We run the model for 10 epochs.
    - The `zero_grad` method is called to reset gradients.
    - Forward pass and loss computation are performed.
    - Backward pass and optimization step are executed.

4. **Time Monitoring**:
    - We record the start and end time to calculate the total time taken for the training process.

In [7]:
import torch.utils.data as data
import time

y_train = torch.tensor(y_train).long()
y_test = torch.tensor(y_test).long()

train_dataset = data.TensorDataset(torch.tensor(X_train), y_train)
train_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=True)

start_time = time.time()

for epoch in range(10):
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

end_time = time.time()
time_taken = end_time - start_time

print(f"Time taken to fit the model: {time_taken:.2f} seconds")


Time taken to fit the model: 9.09 seconds


## Model Evaluation on Test Data

This section focuses on evaluating the trained PyTorch model using the test dataset.

1. **Test Data Loader**: Similar to the training data loader, a test data loader is created using PyTorch's `TensorDataset` and `DataLoader` classes. The batch size is set to 128.

2. **Evaluation Loop**:
    - We disable gradient calculation using `torch.no_grad()` to speed up the computation.
    - Forward passes are performed on test batches to get the model's predictions.
    - Predicted labels are compared with true labels to count the number of correct predictions.

3. **Calculate Test Accuracy**:
    - The accuracy on the test dataset is computed and printed in percentage format with two decimal places.

In [9]:
test_dataset = data.TensorDataset(torch.tensor(X_test), y_test)
test_loader = data.DataLoader(test_dataset, batch_size=128)

correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        output = model(X_batch)
        _, predicted = torch.max(output, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

test_acc = 100 * correct / total
print(f'Test accuracy: {test_acc:.2f}%')


Test accuracy: 96.79%
